In [ ]:
# UCSD Data Science Bootcamp, HW 21 ML
# Alexis Perumal, 4/28/20
# Update sklearn to prevent version mismatches
#
# Model X - Compare Models, no Hyperparameter tuning
#
!pip install sklearn --upgrade

In [ ]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [1]:
import pandas as pd
import numpy as np

# Read the CSV and Perform Basic Data Cleaning

In [2]:
dataset_filename = 'Resources/diagnosis-of-covid-19-and-its-clinical-spectrum.csv'

df = pd.read_csv(dataset_filename)
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# # Drop the null rows
# df = df.dropna()
print(df.shape)
df.head()

(5644, 106)


,patient_id,patient_age_quantile,sars_cov_2_exam_result,patient_addmited_to_regular_ward_1_yes_0_no,patient_addmited_to_semi_intensive_unit_1_yes_0_no,patient_addmited_to_intensive_care_unit_1_yes_0_no,hematocrit,hemoglobin,platelets,mean_platelet_volume,...,hb_saturation_arterial_blood_gases,pco2_arterial_blood_gas_analysis,base_excess_arterial_blood_gas_analysis,ph_arterial_blood_gas_analysis,total_co2_arterial_blood_gas_analysis,hco3_arterial_blood_gas_analysis,po2_arterial_blood_gas_analysis,arteiral_fio2,phosphor,cto2_arterial_blood_gas_analysis
0,44477f75e8169d2,13,negative,f,f,f,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,126e9dd13932f68,17,negative,f,f,f,0.236515,-0.02234,-0.517413,0.010677,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a46b4402a0e5696,8,negative,f,f,f,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,f7d619a94f97c45,5,negative,f,f,f,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,d9e41465789c2b5,15,negative,f,f,f,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.columns

Index(['patient_id', 'patient_age_quantile', 'sars_cov_2_exam_result',
       'patient_addmited_to_regular_ward_1_yes_0_no',
       'patient_addmited_to_semi_intensive_unit_1_yes_0_no',
       'patient_addmited_to_intensive_care_unit_1_yes_0_no', 'hematocrit',
       'hemoglobin', 'platelets', 'mean_platelet_volume',
       ...
       'hb_saturation_arterial_blood_gases',
       'pco2_arterial_blood_gas_analysis',
       'base_excess_arterial_blood_gas_analysis',
       'ph_arterial_blood_gas_analysis',
       'total_co2_arterial_blood_gas_analysis',
       'hco3_arterial_blood_gas_analysis', 'po2_arterial_blood_gas_analysis',
       'arteiral_fio2', 'phosphor', 'cto2_arterial_blood_gas_analysis'],
      dtype='object', length=106)

# Select your features (columns)

In [4]:
# Set features. This will also be used as your x values.
# selected_features = df[['names', 'of', 'selected', 'features', 'here']]
selected_features = df[ ['sars_cov_2_exam_result', 'patient_age_quantile', 'leukocytes', 'platelets', 'monocytes', 'hematocrit', 'eosinophils',
                         'red_blood_cells', 'lymphocytes', 'hemoglobin', 'mean_platelet_volume'] ]
selected_features.head()

,sars_cov_2_exam_result,patient_age_quantile,leukocytes,platelets,monocytes,hematocrit,eosinophils,red_blood_cells,lymphocytes,hemoglobin,mean_platelet_volume
0,negative,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,negative,17,-0.09461,-0.517413,0.357547,0.236515,1.482158,0.102004,0.318366,-0.02234,0.010677
2,negative,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,negative,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,negative,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# # Drop the null rows
print(selected_features.shape)
selected_features = selected_features.dropna()
print(selected_features.shape)

(5644, 11)
(598, 11)


# Create a Train Test Split

Use `koi_disposition` for the y values

In [6]:
# y = df["koi_disposition"].map({'CANDIDATE':0.0, 'CONFIRMED':1.0, 'FALSE POSITIVE':0.0}).values.reshape(-1, 1)
y = selected_features['sars_cov_2_exam_result']
X = selected_features[ ['patient_age_quantile', 'leukocytes', 'platelets', 'monocytes', 'hematocrit', 'eosinophils',
                         'red_blood_cells', 'lymphocytes', 'hemoglobin', 'mean_platelet_volume'] ]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

y

1       negative
8       negative
15      negative
18      negative
22      negative
          ...   
5602    negative
5614    negative
5615    negative
5618    negative
5643    positive
Name: sars_cov_2_exam_result, Length: 598, dtype: object

In [7]:
type(y)

pandas.core.series.Series

In [8]:
total = len(y)
y_pos = len(y.loc[(y=='positive')])
y_neg = len(y.loc[(y=='negative')])
print('Positive outcomes: ', y_pos, ', ', round(y_pos/total*100.0, 1), '%' )
print('Negative outcomes: ', y_neg, ', ', round(y_neg/total*100.0, 1), '%' )

Positive outcomes:  81 ,  13.5 %
Negative outcomes:  517 ,  86.5 %


In [9]:
print("X_train length: ", len(X_train))
print("X_test length: ", len(X_test))
print("y_train length: ", len(y_train))
print("y_test length: ", len(y_test))
print(X_train.shape)
X_train.head()

X_train length:  448
X_test length:  150
y_train length:  448
y_test length:  150
(448, 10)


,patient_age_quantile,leukocytes,platelets,monocytes,hematocrit,eosinophils,red_blood_cells,lymphocytes,hemoglobin,mean_platelet_volume
5581,19,0.609436,0.826701,-0.167718,-2.212562,0.344395,-2.031269,-0.312784,-2.027332,-0.325903
303,15,-0.871009,-1.321369,2.957604,-0.564585,0.386534,-0.215342,-0.295726,-0.460932,0.908221
5172,4,1.257827,-0.165682,-0.482876,-0.862136,-0.835508,-1.202642,0.437772,-0.335620,-0.999063
22,9,-1.132592,-0.668155,2.012129,0.190738,-0.709090,-0.127191,0.002791,-0.147652,1.020415
1317,10,0.492559,0.374475,-1.612194,-0.495919,-0.624811,-0.814774,-0.432190,-0.523588,0.683835


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [10]:
X_train.head()

,patient_age_quantile,leukocytes,platelets,monocytes,hematocrit,eosinophils,red_blood_cells,lymphocytes,hemoglobin,mean_platelet_volume
5581,19,0.609436,0.826701,-0.167718,-2.212562,0.344395,-2.031269,-0.312784,-2.027332,-0.325903
303,15,-0.871009,-1.321369,2.957604,-0.564585,0.386534,-0.215342,-0.295726,-0.460932,0.908221
5172,4,1.257827,-0.165682,-0.482876,-0.862136,-0.835508,-1.202642,0.437772,-0.335620,-0.999063
22,9,-1.132592,-0.668155,2.012129,0.190738,-0.709090,-0.127191,0.002791,-0.147652,1.020415
1317,10,0.492559,0.374475,-1.612194,-0.495919,-0.624811,-0.814774,-0.432190,-0.523588,0.683835


In [11]:
# Scale your data
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)
# y_scaler = StandardScaler().fit(y_train)

In [12]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
# y_train_scaled = y_scaler.transform(y_train)
# y_test_scaled = y_scaler.transform(y_test)

# Setup Evaluation of Multiple Models

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier

In [14]:
lr = LogisticRegression(random_state=99, max_iter=500) # Max_iter=100 (default) generated a warning
rf = RandomForestClassifier(random_state=99)
knn = KNeighborsClassifier(n_neighbors=10)
svm_model = SVC()
sgd = SGDClassifier(shuffle=True)

models = [lr, rf, knn, svm_model, sgd]
names = ['Logistic Regression', 'Random Forest', 'K Nearest Neighbbors', 'Support Vector Classification',
        'Stochastic Gradient Descent']

In [17]:
X_test.iloc[0,:]

patient_age_quantile    15.000000
leukocytes              -0.620558
platelets                0.211172
monocytes                0.672705
hematocrit              -0.152590
eosinophils             -0.540532
red_blood_cells          0.348829
lymphocytes             -0.491893
hemoglobin              -0.398276
mean_platelet_volume    -0.438097
Name: 4533, dtype: float64

In [18]:
selected_features.iloc[0,:]

sars_cov_2_exam_result     negative
patient_age_quantile             17
leukocytes               -0.0946103
platelets                 -0.517413
monocytes                  0.357547
hematocrit                 0.236515
eosinophils                 1.48216
red_blood_cells            0.102004
lymphocytes                0.318366
hemoglobin               -0.0223403
mean_platelet_volume      0.0106766
Name: 1, dtype: object

In [19]:
X_train_scaled[0]

array([ 1.35542951,  0.59227192,  0.9164708 , -0.16499186, -2.18115045,
        0.3212996 , -2.02808174, -0.34011882, -2.00812327, -0.33929236])

In [69]:
for name, model in zip(names, models):
    model.fit(X_train_scaled, y_train)
    score = model.score(X_test_scaled, y_test)
    # probability = model.predict_proba(X_test_scaled)
    # print(name, score, probability)
    print(name, score)

Logistic Regression 0.88
Random Forest 0.88
K Nearest Neighbbors 0.8666666666666667
Support Vector Classification 0.8666666666666667
Stochastic Gradient Descent 0.8


In [42]:
predict_probabilities = lr.predict_proba(X_test_scaled) # Result, % likelihood of neg, # liklihood of pos.

In [54]:
predict_outcomes = lr.predict(X_test_scaled) # Result, % likelihood of neg, # liklihood of pos.

In [55]:
len(probability)

150

In [56]:
X_test.shape

(150, 11)

In [57]:
boo = [x[1] for x in predict_probabilities]
boo


[0.25709433120050385,
 0.6804949166705503,
 0.03805785400979275,
 0.041821758639074086,
 0.00030884058552932897,
 0.07062762978570371,
 0.13792475560687942,
 0.011459007096181684,
 0.012930851934105706,
 0.11607182383487706,
 0.01308991196947183,
 0.014572327149281047,
 0.014122463827032597,
 0.07802243190964889,
 0.02526332375474844,
 0.0014027668118675029,
 0.12783623656228738,
 0.0006271933509374388,
 0.10288501572026043,
 0.004290434050062865,
 0.07421705643168632,
 0.04578213127476796,
 0.004456681634704707,
 0.13028088658871842,
 0.06379022775410832,
 0.02073175427414254,
 0.01010800531901819,
 0.13320854248256334,
 0.013139221978281355,
 0.18240174124615657,
 0.12146294701253181,
 0.01837238470052346,
 0.001757734986012677,
 0.3827352213178404,
 0.07386566527728138,
 0.23299488546320207,
 0.45422882883623533,
 0.014521328809980596,
 0.31614332284118235,
 0.15990748993560097,
 0.29819183831116897,
 0.16120914533576294,
 0.47912837884743337,
 0.002098733172603494,
 0.3644434694350

In [58]:
len(predict_probabilities)

150

In [61]:
X_test2 = X_test
X_test2['postive_prob'] = [x[1] for x in predict_probabilities]
X_test2['prediction'] = predict_outcomes
X_test2['actual outcome'] = y_test
X_test2['correct'] = (predict_outcomes == y_test)
X_test2

/anaconda3/envs/py365_for_class/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/anaconda3/envs/py365_for_class/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/envs/py365_for_class/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

,patient_age_quantile,leukocytes,platelets,monocytes,hematocrit,eosinophils,red_blood_cells,lymphocytes,hemoglobin,mean_platelet_volume,postive_prob,prediction,actual outcome,correct
4533,15,-0.620558,0.211172,0.672705,-0.152590,-0.540532,0.348829,-0.491893,-0.398276,-0.438097,0.257094,negative,negative,True
2699,10,-1.232772,-0.881705,2.826288,0.717175,-0.582671,0.824849,0.028378,0.792188,0.459449,0.680495,positive,negative,False
4086,7,-0.397935,0.751330,0.804021,-0.335699,-0.203417,-0.761883,1.179799,-0.523588,-0.550290,0.038058,negative,negative,True
5016,12,0.275501,0.273981,0.646442,-1.319907,-0.709090,-1.308424,-1.268037,-1.150148,-0.325903,0.041822,negative,positive,False
1560,4,-0.581599,-0.014941,-0.062665,0.877395,3.926242,0.842479,0.778934,0.980156,1.805767,0.000309,negative,negative,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5201,19,-0.406283,-1.635414,-0.141454,-0.495919,-0.709090,-0.973448,0.292779,-0.648900,0.683835,0.328851,negative,positive,False
4244,11,-0.489767,0.550341,-0.220244,-1.182576,-0.835508,-1.185012,-1.447146,-1.024836,0.796029,0.080603,negative,negative,True
385,8,-0.075131,0.035307,-1.717247,0.877395,-0.498393,1.159825,-0.781881,1.168124,0.683835,0.101571,negative,negative,True
2164,6,0.141927,1.718589,-0.561666,1.335167,0.344395,1.477172,0.650998,0.854844,-0.662483,0.013878,negative,negative,True


In [66]:
from sklearn.metrics import confusion_matrix
true_neg, false_pos, false_neg, true_pos = confusion_matrix(y_test, predict_outcomes).ravel()
false_neg
print(true_neg, false_pos, false_neg, true_pos)

125 3 15 7


In [70]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predict_outcomes, target_names=['negative', 'positive']))

              precision    recall  f1-score   support

    negative       0.89      0.98      0.93       128
    positive       0.70      0.32      0.44        22

    accuracy                           0.88       150
   macro avg       0.80      0.65      0.69       150
weighted avg       0.86      0.88      0.86       150



# Save to PKL file

In [ ]:
# See: https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/
import pickle

filename = 'Models/logistic_regression_classifier-alexis.pkl'
pickle.dump(lr, open(filename, 'wb'))


# Test the PKL file

In [ ]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test_scaled, y_test)
print(result)